In [1]:
import os, sys

os.chdir('Petreanu_MEI_generation')
sys.path.append(os.getcwd())

In [2]:
import os
import logging
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy import ndimage
from tqdm.auto import tqdm
import sys
from sensorium.utility.training import read_config
from loaddata.session_info import load_sessions
from utils.imagelib import load_natural_images
from utils.tuning import mean_resp_image
from utils.explorefigs import *
from loaddata.get_data_folder import get_local_drive
import shutil
import torch


# If you want to save a subset of the data, define these manually here. All three variables have to be defined. Else, leave this blank

# session_list = np.array([['LPE10885', '2023_10_20']])
# session_list = np.array(session_list)
# folders = [os.path.join(INPUT_FOLDER, 'LPE10885')]
# files = [[folder, os.path.join(folder, '2023_10_20')]
#          for folder in folders]

# Imports
# -*- coding: utf-8 -*-
"""
This script analyzes neural and behavioral data in a multi-area calcium imaging
dataset with labeled projection neurons. The visual stimuli are natural images.
Matthijs Oude Lohuis, 2023, Champalimaud Center
Anastasia Simonoff, 2024, Bernstein Center for Computational Neuroscience Berlin
"""

# Set working directory to root of repo
current_path = os.getcwd()
# Identify if path has 'Petreanu_MEI_generation' as a folder in it
if 'Petreanu_MEI_generation' in current_path:
    # If so, set the path to the root of the repo
    current_path = current_path.split('Petreanu_MEI_generation')[0] + 'Petreanu_MEI_generation'
else:
    raise FileNotFoundError(
        f'This needs to be run somewhere from within the Petreanu_MEI_generation folder, not {current_path}')
os.chdir(current_path)
sys.path.append(current_path)

run_config = read_config('run_config.yaml') # Must be set
print(run_config)

RUN_NAME = run_config['current_vals']['RUN_NAME'] # MUST be set. Creates a subfolder in the runs folder with this name, containing data, saved models, etc. IMPORTANT: all values in this folder WILL be deleted.
RUN_FOLDER = run_config['current_vals']['RUN_FOLDER']

keep_behavioral_info = run_config['data']['keep_behavioral_info']
area_of_interest = run_config['current_vals']['data']['area_of_interest']
sessions_to_keep = run_config['data']['sessions_to_keep']
INPUT_FOLDER = run_config['data']['INPUT_FOLDER']
OUTPUT_FOLDER = f'{RUN_FOLDER}/data_preprocessed' # relative to molanalysis root folder

if run_config['ASK_FOR_CONFIRMATION']:
    input(f'RUN_NAME: {RUN_NAME}\n\nINPUT FOLDER: {INPUT_FOLDER}\n\nThis will delete all files in the {RUN_FOLDER} folder. Press Enter to continue or Ctrl+C to cancel.')
else:
    print(f'RUN_NAME: {RUN_NAME}\n\nINPUT FOLDER: {INPUT_FOLDER}\n\nThis will delete all files in the {RUN_FOLDER} folder. Automatically continuing...')

# Set up logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(pathname)s - %(message)s', handlers=[logging.StreamHandler()])
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
# Create a StreamHandler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)  # Set the logging level for the handler
# Create a Formatter and attach it to the handler
formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(pathname)s - %(message)s')
console_handler.setFormatter(formatter)
# Add the handler to the logger
logger.addHandler(console_handler)

logger.info('Current working directory: %s', os.getcwd())

# TODO: Fix this so it outputs correctly during figure generation
rmap_logger = logging.getLogger('rastermap')
rmap_logger.setLevel(logging.WARNING)
rmap_logger.addHandler(console_handler)
rmap_logger.propagate = False

# Updated by Anastasia Simonoff for her local computer, etc. This should be updated for your local computer, too.

savedir = os.path.join(get_local_drive(
), 'Users\\asimo\\Documents\\BCCN\\Lab Rotations\\Petreanu Lab\\Figures\\Images' if os.environ['USERDOMAIN'] == 'ULTINTELLIGENCE' else 'OneDrive\\PostDoc\\Figures\\Images\\')
logger.info(f'Saving figures to {savedir}')

# INPUT_FOLDER = '../sensorium/notebooks/data/IM_prezipped'
# Add Add folders two levels deep from INPUT_FOLDER into a list
    
# test if folders already defined 
try: 
    folders
except NameError:
    # First level
    folders = [os.path.join(INPUT_FOLDER, name) for name in os.listdir(
        INPUT_FOLDER) if os.path.isdir(os.path.join(INPUT_FOLDER, name)) and not "merged_data" in name]
    folders = [x.replace("\\", "/") for x in folders]
    # Second level
    files = [[folder, os.path.join(folder, name).replace('\\', '/')] for folder in folders for name in os.listdir(
        folder) if os.path.isdir(os.path.join(folder, name)) and not "merged_data" in name]
    # only get last value after /
    session_list = [[folder.split("/")[-1], name.split("/")[-1]]
                    for folder, name in files]

    # drop ['LPE10919', '2023_11_08'] because the data is not converted yet
    session_list = [x for x in session_list if x != ['LPE10919', '2023_11_08']]
    print(session_list)

if sessions_to_keep != 'all':
    session_list = [x for x in session_list if x in sessions_to_keep]

session_list = np.array(session_list)

# Load one session including raw data: ################################################
# example session with good responses

# Load sessions lazy: (no calciumdata, behaviordata etc.,)
sessions, nSessions = load_sessions(protocol='IM', session_list=session_list, data_folder=INPUT_FOLDER)

# Load proper data and compute average trial responses:
for ises in tqdm(range(nSessions)):    # iterate over sessions

    os.makedirs(os.path.join(OUTPUT_FOLDER, session_list[ises][0], session_list[ises][1], 'data'), exist_ok=True)

    sessions[ises].load_respmat(calciumversion='deconv', keepraw=True)

    # Save respmat
    # np.save(os.path.join(files[ises][1], 'respmat.npy'), sessions[ises].respmat)
    np.save(os.path.join(OUTPUT_FOLDER, session_list[ises][0], session_list[ises][1], 'data', 'respmat.npy'), sessions[ises].respmat)

# Load all IM sessions including raw data: ################################################
# sessions,nSessions   = filter_sessions(protocols = ['IM'])
# for ises in range(nSessions):    # iterate over sessions
#     sessions[ises].load_respmat(calciumversion='deconv',keepraw=False)

def replace_nan_with_avg(arr):
    arr = arr.copy()  # Copy the array to avoid modifying the original
    nan_indices = np.where(np.isnan(arr))[0]  # Get indices of NaN values

    for i in nan_indices:
        # Handle cases where NaN is at the start or end of the array
        if i == 0:
            arr[i] = arr[i + 1]
        elif i == len(arr) - 1:
            arr[i] = arr[i - 1]
        else:
            # Replace NaN with the average of adjacent values
            arr[i] = np.nanmean([arr[i - 1], arr[i + 1]])

    return arr

### Load the natural images:
# natimgdata = load_natural_images(onlyright=True)
natimgdata = load_natural_images()
natimgdata = natimgdata[:, natimgdata.shape[1]//2:, :]  # Only take the right half

2025-02-18 15:11:46,351 - __main__ - INFO - C:\Users\Kerem Sarikaya\AppData\Local\Temp\ipykernel_16148\1881721100.py - Current working directory: d:\Python\Anastasia\Petreanu Lab\Petreanu_MEI_generation
2025-02-18 15:11:46,351 - __main__ - INFO - C:\Users\Kerem Sarikaya\AppData\Local\Temp\ipykernel_16148\1881721100.py - Current working directory: d:\Python\Anastasia\Petreanu Lab\Petreanu_MEI_generation
2025-02-18 15:11:46,353 - __main__ - INFO - C:\Users\Kerem Sarikaya\AppData\Local\Temp\ipykernel_16148\1881721100.py - Saving figures to D:/OneDrive\PostDoc\Figures\Images\
2025-02-18 15:11:46,353 - __main__ - INFO - C:\Users\Kerem Sarikaya\AppData\Local\Temp\ipykernel_16148\1881721100.py - Saving figures to D:/OneDrive\PostDoc\Figures\Images\


{'RUN_NAME': 'test_no_grid_mean_predictor', 'ASK_FOR_CONFIRMATION': False, 'RUN_FOLDER_OVERWRITE': None, 'model': {'init_w_mean_activity': False}, 'data': {'keep_behavioral_info': False, 'area_of_interest': 'V1', 'areas_of_interest': ['V1', 'PM'], 'sessions_to_keep': 'all', 'INPUT_FOLDER': 'D:/Procdata/IM'}, 'MEIs': {'num_meis': 75, 'num_labeled_cells': 10, 'session_id': 'LPE11086', 'session_date': '2024_01_09', 'tier': 'test', 'also_output_to_local': True, 'local_output_folder': 'D:/Bonsai/lab-leopoldo-solene-vr/workflows/MEIs/', 'shape': [68, 135]}, 'dev': {'num_models': 5}, 'current_vals': {'RUN_NAME': 'V1_test_no_grid_mean_predictor', 'RUN_FOLDER': 'runs/V1_test_no_grid_mean_predictor', 'area_id': 0, 'data': {'area_of_interest': 'V1'}}}
RUN_NAME: V1_test_no_grid_mean_predictor

INPUT FOLDER: D:/Procdata/IM

This will delete all files in the runs/V1_test_no_grid_mean_predictor folder. Automatically continuing...
Deleting existing folder runs/V1_test_no_grid_mean_predictor


KeyboardInterrupt: 